# **Installing Ngrok Package**

In [1]:
!pip install pyngrok

# **Mounting Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing Dependencies**

In [3]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import os

# **Loading the Model**

In [6]:
# Changed from '/content/My Drive/my_model.h5' to '/content/drive/MyDrive/my_model.h5'
model_path = '/content/drive/MyDrive/Sign Language AI/Saved Model/my_model.h5'

# Load the model
model = tf.keras.models.load_model(model_path)

# **Loading the saved Scaler and Encoder**

In [7]:
# Load the saved StandardScaler and LabelEncoder
with open('/content/drive/MyDrive/Sign Language AI/Saved Model/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('/content/drive/MyDrive/Sign Language AI/Saved Model/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

In [8]:
# Start Flask app
app = Flask(__name__)

# Set up ngrok to expose the API publicly
ngrok.set_auth_token("2uSiURmoLHTimv0eDPSpfCtezq5_6Dt4SgfeS1RLshnGRpgVR")
public_url = ngrok.connect(5000).public_url
print(f"Public API URL: {public_url}")

# Load correct padding length (Ensure it matches training)
max_length = 32  # Actual training `X_train_padded.shape[1]`

def preprocess_audio(file_path):
    sr = 16000  # Ensure consistent sampling rate
    y, _ = librosa.load(file_path, sr=sr)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T  # Shape: (num_frames, 13)

    # Apply StandardScaler
    mfccs_scaled = scaler.transform(mfccs)

    # Pad to match training input length (32 time steps)
    max_length = 32  # Ensure this matches your model's expected input
    mfccs_padded = pad_sequences([mfccs_scaled], maxlen=max_length, padding='post')  # Shape: (1, 32, 13)

    # Reshape correctly for model input (batch_size, time_steps, features)
    return np.array(mfccs_padded, dtype=np.float32)  # Shape: (1, 32, 13)

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    # Save uploaded audio file
    audio_file = request.files['file']
    file_path = 'temp_audio.wav'
    audio_file.save(file_path)

    # Preprocess the audio
    input_data = preprocess_audio(file_path)

    # Make prediction
    prediction = model.predict(input_data)
    predicted_index = np.argmax(prediction, axis=1)[0]

    # Convert index back to label using trained LabelEncoder
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]

    # Return JSON response
    return jsonify({'prediction': predicted_label})

# Run Flask app
if __name__ == '__main__':
    app.run()


Public API URL: https://92d6-34-32-185-131.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Apr/2025 10:47:32] "GET /predict HTTP/1.1" 405 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [21/Apr/2025 10:47:57] "POST /predict HTTP/1.1" 200 -
